In [41]:
%cd ..

/Users/mateoibarguen/Desktop/CSC 482/complementary_fact_finder


In [68]:
import re
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from gensim.corpora import WikiCorpus, MmCorpus, Dictionary
import pickle

In [16]:
# wordnet_lemmatizer.lemmatize('chsdfarks')

In [34]:
example_sentence = "====Aprill is my favoritte wmonth. "

In [35]:
stop_words = set(stopwords.words('english'))
wordnet_lemmatizer = WordNetLemmatizer()
def tokenize_sentence(sentence, stop_words, lemmatizer=None):
    sentence = re.sub(r'[^\w\s]', '', sentence.lower())
    tokens = [w for w in word_tokenize(sentence) if not w in stop_words and len(w) > 2]
    if lemmatizer:
        tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens]
    return tokens

In [36]:
tokenize_sentence(example_sentence, stop_words, wordnet_lemmatizer)

['aprill', 'favoritte', 'wmonth']

In [73]:
class LoadWikiCorpus:
    def __init__(self):
        self.stop_words = set(stopwords.words('english'))
        self.lemmatizer =  WordNetLemmatizer()
        self.wiki = wiki = WikiCorpus('data/simplewiki-latest-pages-articles.xml.bz2', tokenizer_func=tokenize)
        self.wiki.metadata = True
        self.sents = self.get_sents(1000)
        self.save_list(self.sents, 'data/sents.pkl')
        self.token_sents = self.tokenize_sents(self.sents)
        self.save_list(self.token_sents, 'data/token_sents.pkl')
        
    def get_first_paragraph(self, sents):
        s = []
        i = 0
        while (sents[i][0] != '=') and (i < len(sents) - 1):
            s.append(sents[i])
            i += 1
        return s 
    
    def get_sents(self, num_articles):
        s = []
        i = 0
        for t in self.wiki.get_texts():
            title = t[1][1]
            content = t[0]
            sents = self.get_first_paragraph(content)
            clean_sents = [re.sub(r'[^\w\s]', '', sent) + '.' for sent in sents]
            s.extend([(title, sent) for sent in clean_sents])
            i += 1
            if i > num_articles:
                break
        return s
    
    def tokenize_sentence(self, sentence, stop_words, lemmatizer=None):
        sentence = re.sub(r'[^\w\s]', '', sentence.lower())
        tokens = [w for w in word_tokenize(sentence) if not w in stop_words and len(w) > 2]
        if lemmatizer:
            tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens]
        return tokens
    
    def tokenize_sents(self, sents):
        return [(self.tokenize_sentence(s[1], self.stop_words, self.lemmatizer)) for s in sents]
    
    def save_list(self, list_input, output_path):
        with open(output_path, 'wb') as fb:
            pickle.dump(list_input, fb)
        
        
    

In [74]:
wiki_corpus = LoadWikiCorpus()

In [37]:
def tokenize(text, token_min_len, token_max_len, lower):
    # override original method in wikicorpus.py
    return [token for token in sent_tokenize(text)
            if len(token) >= 20 and not token.startswith('_')]

In [42]:
wiki = WikiCorpus('data/simplewiki-latest-pages-articles.xml.bz2', tokenizer_func=tokenize, processes=7)
wiki.metadata = True

In [43]:
wiki_text = wiki.get_texts()
sents = []
length = 1
i = 0
for t in wiki_text:
    title = t[1][1]
    sents_topic = t[0]
    sents.extend(sents_topic)
    i += 1
    if i > length:
        break

In [45]:
def get_first_paragraph(sents):
    s = []
    i = 0
    while (sents[i][0] != '=') and i < len(sents):
        s.append(sents[i])
        i += 1
    return s

In [61]:
def get_sents(num_articles):
    s = []
    i = 0
    for t in wiki.get_texts():
        title = t[1][1]
        content = t[0]
        sents = get_first_paragraph(content)
        clean_sents = [re.sub(r'[^\w\s]', '', sent) + '.' for sent in sents]
        s.extend([(title, sent) for sent in clean_sents])
        i += 1
        if i > num_articles:
            break
    return s

In [62]:
get_sents(10)

[('April',
  '\nApril is the 4th month of the year and comes between March and May.'),
 ('April', 'It is one of four months to have 30 days.'),
 ('April',
  'April always begins on the same day of week as July and additionally January in leap years.'),
 ('April', 'April always ends on the same day of the week as December.'),
 ('April', 'Aprils flowers are the Sweet Pea and Daisy.'),
 ('April', 'Its birthstone is the diamond.'),
 ('April', 'The meaning of the diamond is innocence.'),
 ('August',
  '\nAugust Aug is the 8th month of the year in the Gregorian calendar coming between July and September.'),
 ('August',
  'It has 31 days the same number of days as the previous month July and is named after Roman Emperor Augustus Caesar.'),
 ('August',
  'August doesnt begin on the same day of the week as any other month in common years but begins on the same day of the week as February in leap years.'),
 ('August', 'August always ends on the same day of the week as November.'),
 ('Art', '\n\n

In [46]:
get_first_paragraph(sents)

["\n'''April''' is the 4th month of the year, and comes between March and May.",
 'It is one of four months to have 30 days.',
 'April always begins on the same day of week as July, and additionally, January in leap years.',
 'April always ends on the same day of the week as December.',
 "April's flowers are the Sweet Pea and Daisy.",
 'Its birthstone is the diamond.',
 'The meaning of the diamond is innocence.']

In [75]:
with open ('data/token_sents.pkl', 'rb') as fp:
    itemlist = pickle.load(fp)

In [77]:
itemlist

[['april', '4th', 'month', 'year', 'come', 'march', 'may'],
 ['one', 'four', 'month', 'day'],
 ['april',
  'always',
  'begin',
  'day',
  'week',
  'july',
  'additionally',
  'january',
  'leap',
  'year'],
 ['april', 'always', 'end', 'day', 'week', 'december'],
 ['april', 'flower', 'sweet', 'pea', 'daisy'],
 ['birthstone', 'diamond'],
 ['meaning', 'diamond', 'innocence'],
 ['august',
  'aug',
  '8th',
  'month',
  'year',
  'gregorian',
  'calendar',
  'coming',
  'july',
  'september'],
 ['day',
  'number',
  'day',
  'previous',
  'month',
  'july',
  'named',
  'roman',
  'emperor',
  'augustus',
  'caesar'],
 ['august',
  'doesnt',
  'begin',
  'day',
  'week',
  'month',
  'common',
  'year',
  'begin',
  'day',
  'week',
  'february',
  'leap',
  'year'],
 ['august', 'always', 'end', 'day', 'week', 'november'],
 ['painting', 'renoir', 'work', 'art'],
 ['art',
  'craft',
  'creative',
  'activity',
  'express',
  'imaginative',
  'technical',
  'skill',
  'produce',
  'artifact